In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
os.chdir('/Users/quangnguyen/Desktop/hcl/project/')
DATA_PATH = './traffic light data/'
os.listdir(DATA_PATH)

['sample-nightClip1',
 '.DS_Store',
 'daySequence2',
 'nightSequence1',
 'nightTrain',
 'Annotations',
 'daySequence1',
 'sample-dayClip6',
 'dayTrain',
 'nightSequence2']

In [3]:
DAY_TRAIN_PATH = './traffic light data/Annotations/Annotations/dayTrain'
# NIGHT_TRAIN_PATH = './traffic light data/Annotations/Annotations/nightTrain'

In [4]:
os.listdir(DAY_TRAIN_PATH)

['dayClip9',
 'dayClip7',
 'dayClip12',
 'dayClip1',
 'dayClip6',
 'dayClip8',
 'dayClip13',
 'dayClip11',
 'dayClip3',
 'dayClip4',
 'dayClip10',
 'dayClip5',
 'dayClip2']

In [5]:
# os.listdir(NIGHT_TRAIN_PATH)

In [6]:
train_day = []
# for clipName in tqdm(sorted(os.listdir(DAY_TRAIN_PATH))):
#     if 'dayClip' not in clipName:
#         continue
#     df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,clipName,'frameAnnotationsBOX.csv'),sep=';')
#     train_day.append(df)
    
# train_day_df = pd.concat(train_day,axis=0)

# df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'dayClip1','frameAnnotationsBOX.csv'),sep=';')
# df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'dayClip2','frameAnnotationsBOX.csv'),sep=';')
df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'dayClip3','frameAnnotationsBOX.csv'),sep=';')


# train_day_df['isNight'] = 0
    
# train_night = []
# for clipName in tqdm(sorted(os.listdir(NIGHT_TRAIN_PATH))):
#     if 'nightClip' not in clipName:
#         continue
#     df = pd.read_csv(os.path.join(NIGHT_TRAIN_PATH,clipName,'frameAnnotationsBOX.csv'),sep=';')
#     train_night.append(df)

# train_night_df = pd.concat(train_night,axis=0)
# train_night_df['isNight'] = 1

# df = pd.concat([train_day_df,train_night_df],axis=0)

In [7]:
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,dayTraining/dayClip3--00001.jpg,warning,622,361,637,379,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,1,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,1
1,dayTraining/dayClip3--00002.jpg,warning,622,358,637,380,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,2,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,2
2,dayTraining/dayClip3--00003.jpg,warning,506,382,518,404,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,3,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,3
3,dayTraining/dayClip3--00004.jpg,warning,620,358,638,380,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4
4,dayTraining/dayClip3--00004.jpg,warning,504,382,519,404,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,4
...,...,...,...,...,...,...,...,...,...,...
1126,dayTraining/dayClip3--00603.jpg,go,578,0,644,102,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,603,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,603
1127,dayTraining/dayClip3--00604.jpg,go,578,0,638,92,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,604,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,604
1128,dayTraining/dayClip3--00605.jpg,go,574,0,640,76,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,605,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,605
1129,dayTraining/dayClip3--00606.jpg,go,570,0,636,56,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,606,dayTraining/dayClip3/dayClip3Shutter0.000800-G...,606


In [8]:
# Duplicate Columns
np.all(df['Origin file'] == df['Origin track']), np.all(df['Origin frame number'] == df['Origin track frame number'])

(True, True)

In [9]:
# Dropping duplicate columns & "Origin file" as we don't need it
df = df.drop(['Origin file','Origin track','Origin track frame number'],axis=1)
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number
0,dayTraining/dayClip3--00001.jpg,warning,622,361,637,379,1
1,dayTraining/dayClip3--00002.jpg,warning,622,358,637,380,2
2,dayTraining/dayClip3--00003.jpg,warning,506,382,518,404,3
3,dayTraining/dayClip3--00004.jpg,warning,620,358,638,380,4
4,dayTraining/dayClip3--00004.jpg,warning,504,382,519,404,4
...,...,...,...,...,...,...,...
1126,dayTraining/dayClip3--00603.jpg,go,578,0,644,102,603
1127,dayTraining/dayClip3--00604.jpg,go,578,0,638,92,604
1128,dayTraining/dayClip3--00605.jpg,go,574,0,640,76,605
1129,dayTraining/dayClip3--00606.jpg,go,570,0,636,56,606


In [10]:
print("Number of Unique Images: ",df.Filename.nunique(),'/',df.shape[0])

Number of Unique Images:  547 / 1131


In [11]:
def changeFilename(x):
    filename = x.Filename
    splitted = filename.split('/')
    clipName = splitted[-1].split('--')[-1]
    return os.path.join(clipName)

df['Filename'] = df.apply(changeFilename, axis=1)

In [12]:
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number
0,00001.jpg,warning,622,361,637,379,1
1,00002.jpg,warning,622,358,637,380,2
2,00003.jpg,warning,506,382,518,404,3
3,00004.jpg,warning,620,358,638,380,4
4,00004.jpg,warning,504,382,519,404,4
...,...,...,...,...,...,...,...
1126,00603.jpg,go,578,0,644,102,603
1127,00604.jpg,go,578,0,638,92,604
1128,00605.jpg,go,574,0,640,76,605
1129,00606.jpg,go,570,0,636,56,606


In [13]:
# We will change annotations to only -> stop (RED), go (GREEN) & warning (YELLOW)
label_to_idx = {'go':1, 'warning':2, 'stop': 3}
idx_to_label = {v:k for k,v in label_to_idx.items()}

def changeAnnotation(x):
    if 'go' in x['Annotation tag']:
        return label_to_idx['go']
    elif 'warning' in x['Annotation tag']:
        return label_to_idx['warning']
    elif 'stop' in x['Annotation tag']:
        return label_to_idx['stop']
    
df['Annotation tag'] = df.apply(changeAnnotation,axis=1)

annotation_tags = df['Annotation tag'].unique()
annotation_tags

array([2, 3, 1])

In [14]:
df.head()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number
0,00001.jpg,2,622,361,637,379,1
1,00002.jpg,2,622,358,637,380,2
2,00003.jpg,2,506,382,518,404,3
3,00004.jpg,2,620,358,638,380,4
4,00004.jpg,2,504,382,519,404,4


In [15]:
# Changing Column Names
df.columns = ['image_id','label','x_min','y_min','x_max','y_max','frame']
df

,image_id,label,x_min,y_min,x_max,y_max,frame
0,00001.jpg,2,622,361,637,379,1
1,00002.jpg,2,622,358,637,380,2
2,00003.jpg,2,506,382,518,404,3
3,00004.jpg,2,620,358,638,380,4
4,00004.jpg,2,504,382,519,404,4
...,...,...,...,...,...,...,...
1126,00603.jpg,1,578,0,644,102,603
1127,00604.jpg,1,578,0,638,92,604
1128,00605.jpg,1,574,0,640,76,605
1129,00606.jpg,1,570,0,636,56,606


In [16]:
print("Number of Unique Images: ",df.image_id.nunique(),'/',df.shape[0])

Number of Unique Images:  547 / 1131


In [17]:
df

,image_id,label,x_min,y_min,x_max,y_max,frame
0,00001.jpg,2,622,361,637,379,1
1,00002.jpg,2,622,358,637,380,2
2,00003.jpg,2,506,382,518,404,3
3,00004.jpg,2,620,358,638,380,4
4,00004.jpg,2,504,382,519,404,4
...,...,...,...,...,...,...,...
1126,00603.jpg,1,578,0,644,102,603
1127,00604.jpg,1,578,0,638,92,604
1128,00605.jpg,1,574,0,640,76,605
1129,00606.jpg,1,570,0,636,56,606


## Rename all images in the dataset

In [21]:
os.getcwd()

'/Users/quangnguyen/Desktop/hcl/project'

In [22]:
collection = '/Users/quangnguyen/Desktop/hcl/project/datasets/train/images/'
# collection = '/Users/quangnguyen/Desktop/hcl/project/data/val/images/'
# collection = '/Users/quangnguyen/Desktop/hcl/project/data/test/images/'
print(len(os.listdir(collection)))

2162


In [24]:
import os
os.getcwd()

for i, filename in enumerate(sorted(os.listdir(collection))):
    new_filename = filename.split('--')[-1]
    os.rename(collection + filename, collection + new_filename)

## reformat labeled data

In [49]:
## Write dataframe to textfile

os.chdir('/Users/quangnguyen/Desktop/hcl/project/datasets/train/labels')
# os.chdir('/Users/quangnguyen/Desktop/hcl/project/data/val/labels')
# os.chdir('/Users/quangnguyen/Desktop/hcl/project/data/test/labels')
for index, row in df.iterrows():
    print(row)
    filename = row['image_id'].split('.')[0] + '.txt'
    xmin = float(row['x_min'])
    ymin = float(row['y_min'])
    xmax = float(row['x_max'])
    ymax = float(row['y_max'])
    
    width = xmax - xmin
    height = ymax - ymin
    x_center = xmin + ((xmax - xmin) / 2)
    y_center = ymin + ((ymax - ymin) / 2 )


    print(0, x_center, y_center, width, height)

    with open(filename, 'a') as outfile:
        outfile.write(f'0 {x_center} {y_center} {width} {height}\n')
    

image_id    00001.jpg
label               2
x_min             622
y_min             361
x_max             637
y_max             379
frame               1
Name: 0, dtype: object
0 629.5 370.0 15.0 18.0
image_id    00002.jpg
label               2
x_min             622
y_min             358
x_max             637
y_max             380
frame               2
Name: 1, dtype: object
0 629.5 369.0 15.0 22.0
image_id    00003.jpg
label               2
x_min             506
y_min             382
x_max             518
y_max             404
frame               3
Name: 2, dtype: object
0 512.0 393.0 12.0 22.0
image_id    00004.jpg
label               2
x_min             620
y_min             358
x_max             638
y_max             380
frame               4
Name: 3, dtype: object
0 629.0 369.0 18.0 22.0
image_id    00004.jpg
label               2
x_min             504
y_min             382
x_max             519
y_max             404
frame               4
Name: 4, dtype: object
0 511.5 393.0 15.0 

## Training

In [50]:
train_path = '/Users/quangnguyen/Desktop/hcl/project/datasets/train/images'
val_path = '/Users/quangnguyen/Desktop/hcl/project/datasets/val/images'
test_path = '/Users/quangnguyen/Desktop/hcl/project/datasets/test/'

## Configure yaml file

In [53]:
os.chdir('/Users/quangnguyen/Desktop/hcl/project/')


In [55]:

%cd yolov5/

[Errno 2] No such file or directory: 'yolov5/'
/Users/quangnguyen/Desktop/hcl/project/yolov5


In [56]:
!pwd

/Users/quangnguyen/Desktop/hcl/project/yolov5


In [57]:
!python train.py --img 416 --batch 8 --epochs 50 --data /Users/quangnguyen/Desktop/hcl/project/datasets/dataset.yaml --weights /Users/quangnguyen/Desktop/hcl/project/yolov5s.pt --nosave --cache

train: weights=/Users/quangnguyen/Desktop/hcl/project/yolov5s.pt, cfg=, data=/Users/quangnguyen/Desktop/hcl/project/datasets/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=416, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (371/371), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 745 (delta 369), reused 364 (delta 364), pack-reused 374
Receiving objects: 100% (745/745), 187.49 KiB | 404.00 KiB/s, done.
Resolving deltas: 100% (561/561), completed with 39 local objects.
From https